In [4]:
import pandas as pd
import nltk
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer # to perfor bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perfor bow
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perform cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

# Download nltk data libraries. All can be downloaded by using nltk.download('all')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# nltk.download('all')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_csv('/kaggle/input/indian-government-schemes/updated_data.csv')

df.head(20) # See first 20 lines

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,Unnamed: 9,tags
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...",NaN,"Missing, Fisherman, Relief, Financial Assistan..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,NaN,"Trainings, Financial Assistance, AICTE"
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,NaN,"Building Worker, Construction Workers, Unregis..."
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,NaN,"Goods And Services Marketing, Turnkey Projects..."
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,NaN,"Social Welfare, Financial Assistance, Deceased..."
5,Incentive For The Intra Caste Marriage within ...,ifticmwstc,"The scheme ""Incentive for Intra Caste Marriage...","An incentive of ₹2,00,000/- is provided to the...",The couple should be residing in Karnataka. Bo...,Step 1: Visit the official website of the Trib...,Aadhaar Card Bank Account Details Marriage Pro...,State,Social welfare & Empowerment,NaN,"Intra Caste, Marriage, Schedule Tribe, Incenti..."
6,Incentive Scheme for MSMEs in Powerloom Sector...,ismpsscis,The scheme “State Capital Investment Subsidy” ...,20% of Fixed Capital Investment on Plant and m...,The Scheme shall be generally applicable to al...,"A micro, small or medium enterprise in Powerlo...",A copy of the Memorandum of Association and Ar...,State,Business & Entrepreneurship,NaN,"Powerloom, Incentives, State Capital Investmen..."
7,Indira Mahila Shakti Udyam Protsahan Yojana,imspesy,The Indira Mahila Shakti Udyam Protsahan Yojan...,"Loan Limit Loan amount of ₹ 50,00,000/- to ind...",The applicant's age should be 18 years or more...,Registration Step-1: Applicant have to visit t...,Aadhar card. Address proof. Bank account state...,State,Women and Child,NaN,"Women, Empowerment, Development, Motivaction"
8,Mukhyamantri Shramik Aujaar Sahayata Yojana,msasy,The Labor Department of Chhattisgarh launched ...,Tool kits can given free of cost to the regist...,The Applicant must be a native of Chhattisgarh...,Registration of a Construction Worker Under C...,Bank passbook photocopy. Self-declaration cert...,State,Social welfare & Empowerment,NaN,"Financial Assistance, Construction Workers, La..."
9,Nirman Shramik Jeevan va Bhavishya Suraksha Yo...,nsjbsy,"Launched in 2016, the scheme ""Nirman Shramik J...",Pradhan Mantri Suraksha Bima Yojana (PMSBY) 10...,1. The applicant must be registered as a bene...,REGISTRATION Step 1: Visit the Official Portal...,Copy of Beneficiary Registration Identity C

In [7]:
df.shape[0] # Returns the number of rows in dataset

3400

In [8]:
df.isnull().sum()

scheme_name          0
slug                 0
details              0
benefits             0
eligibility          0
application          2
documents           11
level                0
schemeCategory       0
Unnamed: 9        3400
tags                29
dtype: int64

# Filling or handling the missing values

In [9]:
df.ffill(axis = 0,inplace=True) # fills the null value with the previous value.
df.head(20)

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,Unnamed: 9,tags
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...",NaN,"Missing, Fisherman, Relief, Financial Assistan..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,NaN,"Trainings, Financial Assistance, AICTE"
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,NaN,"Building Worker, Construction Workers, Unregis..."
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,NaN,"Goods And Services Marketing, Turnkey Projects..."
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,NaN,"Social Welfare, Financial Assistance, Deceased..."
5,Incentive For The Intra Caste Marriage within ...,ifticmwstc,"The scheme ""Incentive for Intra Caste Marriage...","An incentive of ₹2,00,000/- is provided to the...",The couple should be residing in Karnataka. Bo...,Step 1: Visit the official website of the Trib...,Aadhaar Card Bank Account Details Marriage Pro...,State,Social welfare & Empowerment,NaN,"Intra Caste, Marriage, Schedule Tribe, Incenti..."
6,Incentive Scheme for MSMEs in Powerloom Sector...,ismpsscis,The scheme “State Capital Investment Subsidy” ...,20% of Fixed Capital Investment on Plant and m...,The Scheme shall be generally applicable to al...,"A micro, small or medium enterprise in Powerlo...",A copy of the Memorandum of Association and Ar...,State,Business & Entrepreneurship,NaN,"Powerloom, Incentives, State Capital Investmen..."
7,Indira Mahila Shakti Udyam Protsahan Yojana,imspesy,The Indira Mahila Shakti Udyam Protsahan Yojan...,"Loan Limit Loan amount of ₹ 50,00,000/- to ind...",The applicant's age should be 18 years or more...,Registration Step-1: Applicant have to visit t...,Aadhar card. Address proof. Bank account state...,State,Women and Child,NaN,"Women, Empowerment, Development, Motivaction"
8,Mukhyamantri Shramik Aujaar Sahayata Yojana,msasy,The Labor Department of Chhattisgarh launched ...,Tool kits can given free of cost to the regist...,The Applicant must be a native of Chhattisgarh...,Registration of a Construction Worker Under C...,Bank passbook photocopy. Self-declaration cert...,State,Social welfare & Empowerment,NaN,"Financial Assistance, Construction Workers, La..."
9,Nirman Shramik Jeevan va Bhavishya Suraksha Yo...,nsjbsy,"Launched in 2016, the scheme ""Nirman Shramik J...",Pradhan Mantri Suraksha Bima Yojana (PMSBY) 10...,1. The applicant must be registered as a bene...,REGISTRATION Step 1: Visit the Official Portal...,Copy of Beneficiary Registration Identity C

In [10]:
df = df.drop(columns=["Unnamed: 9"])

# Checking if still missing values persist or not

In [11]:
df.isnull().sum()

scheme_name       0
slug              0
details           0
benefits          0
eligibility       0
application       0
documents         0
level             0
schemeCategory    0
tags              0
dtype: int64

# Creating a new dataset

In [12]:
df1 =df.head(10)
df1

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,tags
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...","Missing, Fisherman, Relief, Financial Assistan..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,"Trainings, Financial Assistance, AICTE"
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,"Building Worker, Construction Workers, Unregis..."
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,"Goods And Services Marketing, Turnkey Projects..."
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,"Social Welfare, Financial Assistance, Deceased..."
5,Incentive For The Intra Caste Marriage within ...,ifticmwstc,"The scheme ""Incentive for Intra Caste Marriage...","An incentive of ₹2,00,000/- is provided to the...",The couple should be residing in Karnataka. Bo...,Step 1: Visit the official website of the Trib...,Aadhaar Card Bank Account Details Marriage Pro...,State,Social welfare & Empowerment,"Intra Caste, Marriage, Schedule Tribe, Incenti..."
6,Incentive Scheme for MSMEs in Powerloom Sector...,ismpsscis,The scheme “State Capital Investment Subsidy” ...,20% of Fixed Capital Investment on Plant and m...,The Scheme shall be generally applicable to al...,"A micro, small or medium enterprise in Powerlo...",A copy of the Memorandum of Association and Ar...,State,Business & Entrepreneurship,"Powerloom, Incentives, State Capital Investmen..."
7,Indira Mahila Shakti Udyam Protsahan Yojana,imspesy,The Indira Mahila Shakti Udyam Protsahan Yojan...,"Loan Limit Loan amount of ₹ 50,00,000/- to ind...",The applicant's age should be 18 years or more...,Registration Step-1: Applicant have to visit t...,Aadhar card. Address proof. Bank account state...,State,Women and Child,"Women, Empowerment, Development, Motivaction"
8,Mukhyamantri Shramik Aujaar Sahayata Yojana,msasy,The Labor Department of Chhattisgarh launched ...,Tool kits can given free of cost to the regist...,The Applicant must be a native of Chhattisgarh...,Registration of a Construction Worker Under C...,Bank passbook photocopy. Self-declaration cert...,State,Social welfare & Empowerment,"Financial Assistance, Construction Workers, La..."
9,Nirman Shramik Jeevan va Bhavishya Suraksha Yo...,nsjbsy,"Launched in 2016, the scheme ""Nirman Shramik J...",Pradhan Mantri Suraksha Bima Yojana (PMSBY) 10...,1. The applicant must be registered as a bene...,REGISTRATION Step 1: Visit the Official Portal...,Copy of Beneficiary Registration Identity Card...,State,Health & Wellness,"Medical, Insura

In [13]:
# function that converts text into lower case removes special characters

def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [14]:
step1(df1['scheme_name'])

 immediate relief assistance  under  welfare and relief for fishermen during lean seasons and natural calamities scheme 
aicte short term training programme sfurti scheme
burial and ex gratia payment scheme in case of death of unregistered laborer during work at construction site
consortia   tender marketing scheme
garuda scheme for funeral expense
incentive for the intra caste marriage within scheduled tribe community
incentive scheme for msmes in powerloom sector  state capital investment subsidy
indira mahila shakti udyam protsahan yojana
mukhyamantri shramik aujaar sahayata yojana
nirman shramik jeevan va bhavishya suraksha yojana


In [15]:
nltk.download('punkt_tab')
# word tokenizing

s = 'aicte short term training programme sfurti scheme'
words = word_tokenize(s)
print(words)

['aicte', 'short', 'term', 'training', 'programme', 'sfurti', 'scheme']


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Lemmatization

In [16]:
lemma = wordnet.WordNetLemmatizer() # initializing lemmatizer

lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [17]:
nltk.download('averaged_perceptron_tagger_eng')
pos_tag(nltk.word_tokenize(s), tagset=None) # returns the parts of speech of ever

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


[('aicte', 'RB'),
 ('short', 'JJ'),
 ('term', 'NN'),
 ('training', 'VBG'),
 ('programme', 'JJ'),
 ('sfurti', 'JJ'),
 ('scheme', 'NN')]

In [18]:
# function that performs text normalization steps

def text_normalization(text):
    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # initializing lemmatization
    tags_list=pos_tag(tokens,tagset=None)
    lema_words=[] # empty list
    for token, pos_token in tags_list:
        if pos_token.startswith('V'): # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adhactive
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list

    return " ".join(lema_words) # returns the lemmatized tokens as a sentence

In [19]:
text_normalization('garudas scheme for funeral expense')

'garuda scheme for funeral expense'

In [20]:
df['lemmatized_text'] = df['scheme_name'].apply(text_normalization)
df.tail(20)

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,tags,lemmatized_text
3380,YSR Vahana Mitra,ysrvm,"Launched on 1st June 2021, ""YSR Vahana Mitra"" ...","In this scheme, the government provides financ...","1.\tThe applicant shall ""own"" and ""drive"" an A...",Step 1: On Citizen Schemes Portal landing page...,Aadhaar Number of Self. Aadhaar Number of Fami...,State,"Transport & Infrastructure, Social welfare & E...","Financial Asisstance, Social Welfare, Transpor...",ysr vahana mitra
3381,Yashwantrao Chavan Mukta Vasahat Yojana for VJNTs,ycmvyfvjnts,"The scheme ""Yashwantrao Chavan Mukta Vasahat Y...",Each family is provided with a plot of land me...,The family of the applicant should be residing...,Step 1: The interested applicant should visit...,"Identity Proof (Aadhaar Card, Voter Card, etc....",State,"Social welfare & Empowerment, Housing & Shelte...","Vimukta Jati, Nomadic Tribe, Family, Financial...",yashwantrao chavan mukta vasahat yojana for vjnts
3382,Yatra Pass in DTC non AC Buses for the Constru...,ypidnabftcw,"The ""Yatra Pass in DTC non AC Buses for the Co...",A pass of 2400/- per three months for non AC D...,For Registration The applicant should be a res...,Registration Step 1: Visit the DBoCWWB website...,Labour card. Any other documents as directed b...,State,"Social welfare & Empowerment, Transport & Infr...","Yatra Pass, Labour, Construction Worker, Trave...",yatra pas in dtc non ac bus for the constructi...
3383,Young Innovators Program,yipjk,Innovation may be defined as exploiting new id...,The total amount of assistance provided under ...,Applicant must be between the 18-35 years age ...,Applicant must visit the official website. ﻿ F...,﻿ Financial Justification and Stage wise fundi...,State,Social welfare & Empowerment,"Yatra Pass, Labour, Construction Worker, Trave...",young innovator program
3384,Young Interns Program,yip,The Young Interns Program (YIP) is an internsh...,"Stipend ₹ 30,000/- per month. ﻿ Laptop Connect...",General The age of the applicant should be bet...,Application Applications will be invited from ...,Photograph Letter of Intent (in Hindi or Engli...,State,Education & Learning,"Internship, Education, Student, Government",young intern program
3385,Young Investigators Programme in Biotechnology,yipb,The programme “Young Investigator’s Programme ...,Young Scientists without fellowship/salary are...,1.\tThe applicant should possess a Ph.D. in an...,Applications are invited through advertisement...,1.\tIdentity proof of applicant 2.\tPassport s...,State,Education & Learning,"Young Investigator, Biotechnology, Young Scien...",young investigator programme in biotechnology
3386,Youth Hostel Scheme,yhs,The Youth Hostel Scheme is managed by the Depa...,Affordable accommodation: Youth hostels offer ...,There is no age restriction but if applicant i...,Applicant must visit the official website . Cl...,Completed YHAI membership application form Pas...,Central,Housing & Shelter,"Shelter, Youth Hostel, Residence",youth hostel scheme
3387,Yuva Srujan Puraskar (Navsarjan Chetana Puraskar),yspncp,"The scheme ""Yuva Srujan Puraskar (Navsarjan Ch...","A memento, certificate of appreciation and fin...",The applicant should be an individual artist. ...,Step 1: The interested applicant should visit ...,Recommendation Letter From Individual/ Institu...,State,Sports & Culture,"Artist, Culture, Award, Achievement, Financial...",yuva srujan puraskar navsarjan chetana puraskar
3388,Yuvasree Prakalpa,yuvasree,"'Yuvasree' (earlier named as ""Yuva Utsaha Prak...",Cash Benefits: Subject to fulfillment of the a...,The applicant should be a resident of West Ben...,Registration Process : Applicant first visit t...,Annexure I ( Application Form). Annexure -II (...,State,Skills & Employment,"Employment, Job Seeker, Financial Assistance, ...",yuvasree prakalpa
3389,Zero Budget Natural Farming,zbnf,Zero Budget Natural Farming is natural farming...,A subsidy of 50% provides to farmers 

In [21]:
# all the stop words we have

stop = stopwords.words('english')
stop

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

# bag of words

In [22]:
cv = CountVectorizer()  # initializing the count vectorizer

X = cv.fit_transform(df['lemmatized_text']).toarray()

X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
# returns all the unique word from data

features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,aadhar,aadharbhut,aadharit,aadim,aadmi,aahaar,aahar,aajeevika,aaloo,aam,...,youth,yozana,ysr,yuba,yuva,yuvak,yuvasree,zed,zero,zila
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Question = 'Can you tell me how you help people?' # considering an example

In [25]:
# checking for stop words

Q = []
a = Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b = " ".join(Q)

b # print the values

'Can tell help people?'

In [26]:
Question_lemma = text_normalization(b) # applyig the function that we created for
Question_bow = cv.transform([Question_lemma]).toarray() # applying bow

In [27]:
Question_bow

array([[0, 0, 0, ..., 0, 0, 0]])

# Now to get the related response we shall find the cosine similaity between the question and the lemmatized text we have.

# similarity

In [28]:
# cosine similarity for the above question we considered.

cosine_value = 1 - pairwise_distances(df_bow, Question_bow, metric = 'cosine')
(cosine_value)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [29]:
df['similarity_bow'] = cosine_value # creating a new column
df

,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,tags,lemmatized_text,similarity_bow
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...","Missing, Fisherman, Relief, Financial Assistan...",immediate relief assistance under welfare and ...,0.0
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,"Trainings, Financial Assistance, AICTE",aicte short term train programmesfurti scheme,0.0
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,"Building Worker, Construction Workers, Unregis...",burial and exgratia payment scheme in case of ...,0.0
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,"Goods And Services Marketing, Turnkey Projects...",consortium tender marketing scheme,0.0
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,"Social Welfare, Financial Assistance, Deceased...",garuda scheme for funeral expense,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3395,“Ishan Uday” Special Scholarship Scheme For No...,iu-sss-ner,A scholarship scheme by the Ministry of Educat...,"1) UGC will award 10,000 (ten thousand) fresh...",The applicant must be a domicile of NER. The p...,New Registration : Step 1: Visit the Registrat...,Domicile certificate to be issued by the compe...,Central,Education & Learning,"Scholarship, Financial Assistance",ishan uday special scholarship scheme for nort...,0.0
3396,“Medical Camps (Allopathy and Indian systems o...,mcacism-giapsca,The sub-scheme “Medical Camps (Allopathy and I...,Free Health Checkup and Free Medicines. Awaren...,The applicant should be 60 years or above in a...,Step 1: The interested applicant should visit ...,Aadhaar Card. Birth Certificate / Age Proof. R...,State,Health & Wellness,"Medicine, Health, Checkup, Senior Citizen, Doctor",medical camp allopathy and indian system of me...,0.0
3397,“RAKSHAK PURASKARS’’ Award,rpa,The Directorate of Women and Child Development...,The recipients of the awarded are given a scro...,The applicant should have shown exceptional co...,Step 1: In the prescribed format of the applic...,Passport Size Photograph (Duly Attested by a G...,State,"Social welfare & Empowerment, Public Safety,La...","Award, Crime, Women, Girl, Violence, Harrassment",rakshak puraskars award,0.0
3398,"“Training in Coir” Component of the ""Developme...",ticcotdocs,The scheme “Training in Coir” Component of the...,"Stipend of ₹1,500 per month per trainee during...",The applicant should be a native of Puducherry...,Step 1: The interested applicant should take p...,Residential Certific

In [31]:
df['Text Response'] = (
    df['slug'].astype(str) + " " +
    df['details'].astype(str) + " " +
    df['benefits'].astype(str) + " " +
    df['eligibility'].astype(str) + " " +
    df['application'].astype(str) + " " +
    df['documents'].astype(str) + " " +
    df['level'].astype(str) + " " +
    df['schemeCategory'].astype(str) + " " +
    df['tags'].astype(str)
)


In [32]:
df_simi = pd.DataFrame(df, columns=['Text Response','similarity_bow']) # taking similarity value of responses for the question we took
df_simi

,Text Response,similarity_bow
0,"ira-wrflsncs The scheme ""Immediate Relief Assi...",0.0
1,astpss Short Term Training Programme-SFURTI Pr...,0.0
2,"baepsicodouldwact Launched in 2014, the "" Buri...",0.0
3,ctms Promotion of the product of Micro and Sma...,0.0
4,gsfe Andhra Pradesh Brahmin Welfare Corporatio...,0.0
...,...,...
3395,iu-sss-ner A scholarship scheme by the Ministr...,0.0
3396,mcacism-giapsca The sub-scheme “Medical Camps ...,0.0
3397,rpa The Directorate of Women and Child Develop...,0.0
3398,ticcotdocs The scheme “Training in Coir” Compo...,0.0


In [33]:
df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False) # sorting the values
df_simi_sort.head()

,Text Response,similarity_bow
591,"chs The ""Coaching Help Scheme"" introduced by t...",0.333333
523,"cmhs The scheme was launched on 02-10-2019, th...",0.288675
610,cctofp The office of the Deputy Director of Ag...,0.235702
592,"chsjgne The scheme ""Coaching Help Scheme for J...",0.235702
1082,fhdfh Financial help on death of family head s...,0.218218


In [34]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold

,Text Response,similarity_bow
591,"chs The ""Coaching Help Scheme"" introduced by t...",0.333333
523,"cmhs The scheme was launched on 02-10-2019, th...",0.288675
610,cctofp The office of the Deputy Director of Ag...,0.235702
592,"chsjgne The scheme ""Coaching Help Scheme for J...",0.235702
1082,fhdfh Financial help on death of family head s...,0.218218
2538,rcmrf-mh Rajasthan Chief Minister’s Relief Fun...,0.218218
1127,fsrpp The scheme “Free Supply of Rice to the P...,0.204124


In [35]:
index_value = cosine_value.argmax() # returns the index number of highest value
index_value 

np.int64(591)

In [36]:
(Question)

'Can you tell me how you help people?'

In [37]:
df['Text Response'].loc[index_value] # The text at the above index becomes the response for the question

'chs The "Coaching Help Scheme" introduced by the Social Justice and Empowerment Department, Gujarat, offers an annual incentive to eligible students for tuition classes outside their school/college. The aim is to help academically bright students continue their studies without financial constraints. \ufeff \ufeff Financial Assistance: ₹15,000/- per year for coaching expenses outside the school/college. \ufeff The applicant must belong to a non-reserved category. The applicant must have scored 70% or more in Standard 10th. The applicant must be currently studying in the science stream in Standard 11th or Standard 12th. The applicant’s annual family income should not exceed ₹4,50,000/-. \ufeff Note: The applicant can avail of the assistance only once per academic year. Registration Process: Step 1: Go to the Corporation\'s official website and navigate to the "Gujarat Unreserved Education and Economical Development Corporation" section under the Corporation menu. Step 2: Click on "Coach

# tf-idf

In [38]:
Question1 = 'What is Pradhan Mantri Awas Yojana?'

In [39]:
tfidf = TfidfVectorizer()
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.fit_transform([Question_lemma1]).toarray() # applying tf-idf

In [40]:
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()

In [41]:
# returns all the unique word from data with a score of that word

df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names_out()) 
df_tfidf.head()

,aadhar,aadharbhut,aadharit,aadim,aadmi,aahaar,aahar,aajeevika,aaloo,aam,...,youth,yozana,ysr,yuba,yuva,yuvak,yuvasree,zed,zero,zila
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df.shape

(3400, 13)

# Similarity

In [43]:
# Split the transformed data back into the document matrix and the question vector
df_tfidf =x_tfidf[:-1]  # All rows except the last (documents)
Question_tfidf =x_tfidf[-1].reshape(1, -1)  # The last row (question)

In [44]:
cos=1-pairwise_distances(df_tfidf,x_tfidf,metric='cosine')  # applying cosine similarity
cos

array([[1.        , 0.01054872, 0.1176969 , ..., 0.        , 0.00967358,
        0.        ],
       [0.01054872, 1.        , 0.01010477, ..., 0.        , 0.01463375,
        0.        ],
       [0.1176969 , 0.01010477, 1.        , ..., 0.        , 0.10006388,
        0.        ],
       ...,
       [0.06919744, 0.        , 0.05206103, ..., 0.        , 0.10417707,
        0.01911639],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00967358, 0.01463375, 0.10006388, ..., 0.        , 1.        ,
        0.        ]])

In [45]:
print(cos.shape)

(3399, 3400)


In [46]:
if len(cos) < len(df):
    cos = np.pad(cos, (0, len(df) - len(cos)), mode='constant', constant_values=0)

In [47]:
cos = cos[:, 0] 
print(cos.shape)


(3400,)


In [48]:
df['similarity_tfidf']= cos.flatten() # creating a new column 
df_simi_tfidf = pd.DataFrame(df, columns=['Text Response','similarity_tfidf']) # taking similarity value of responses for the question we took
df_simi_tfidf 

,Text Response,similarity_tfidf
0,"ira-wrflsncs The scheme ""Immediate Relief Assi...",1.000000
1,astpss Short Term Training Programme-SFURTI Pr...,0.010549
2,"baepsicodouldwact Launched in 2014, the "" Buri...",0.117697
3,ctms Promotion of the product of Micro and Sma...,0.012069
4,gsfe Andhra Pradesh Brahmin Welfare Corporatio...,0.029756
...,...,...
3395,iu-sss-ner A scholarship scheme by the Ministr...,0.021299
3396,mcacism-giapsca The sub-scheme “Medical Camps ...,0.069197
3397,rpa The Directorate of Women and Child Develop...,0.000000
3398,ticcotdocs The scheme “Training in Coir” Compo...,0.009674


In [49]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False) # sorting the values
df_simi_tfidf_sort.head(10)

,Text Response,similarity_tfidf
0,"ira-wrflsncs The scheme ""Immediate Relief Assi...",1.000000
60,"wrflsncs-idra The scheme ""Immediate Death Reli...",0.975879
1719,"wrflsncs-lsa The scheme ""Lean Season Assistanc...",0.856534
50,"wrflsncs-fbps The scheme ""Fishing Ban Period A...",0.778771
2196,ncrfs “Natural Calamity Relief to Fishermen of...,0.577411
1092,"fars The ""Fishermen Accident Relief Scheme"" in...",0.416044
2593,"runcpp-pbaocwwb The ""Relief under Natural Cala...",0.409487
2590,"rdnccff-sw ""Relief During Natural Calamities S...",0.394762
2199,"ndr-mbocwwb The ""Natural Death Relief"" scheme ...",0.357423
2200,"ndr-tbocwwb The ""Natural Death Relief - TBOCWW...",0.349305


In [50]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,Text Response,similarity_tfidf
0,"ira-wrflsncs The scheme ""Immediate Relief Assi...",1.000000
60,"wrflsncs-idra The scheme ""Immediate Death Reli...",0.975879
1719,"wrflsncs-lsa The scheme ""Lean Season Assistanc...",0.856534
50,"wrflsncs-fbps The scheme ""Fishing Ban Period A...",0.778771
2196,ncrfs “Natural Calamity Relief to Fishermen of...,0.577411
1092,"fars The ""Fishermen Accident Relief Scheme"" in...",0.416044
2593,"runcpp-pbaocwwb The ""Relief under Natural Cala...",0.409487
2590,"rdnccff-sw ""Relief During Natural Calamities S...",0.394762
2199,"ndr-mbocwwb The ""Natural Death Relief"" scheme ...",0.357423
2200,"ndr-tbocwwb The ""Natural Death Relief - TBOCWW...",0.349305


In [51]:
index_value1 = cos.argmax() # returns the index number of highest value
index_value1

np.int64(0)

In [52]:
Question1

'What is Pradhan Mantri Awas Yojana?'

In [53]:
df['Text Response'].loc[index_value1]  # returns the text at that index

'ira-wrflsncs The scheme "Immediate Relief Assistance" is a Sub-Component under the scheme "Welfare and Relief for Fishermen During Lean Seasons and Natural Calamities Scheme". The scheme is extended to all the regions of the Union territory of Puducherry. The scheme is introduced with the objective of extending financial assistance to the fishermen\'s families to compensate for the loss due to the missing breadwinner and to support them financially to run their family. ₹ 1,00,000, in two installments of ₹ 50,000 each, as immediate relief assistance for the family (legal heir) of the missing fisherman. \ufeff Disbursal Initially, 50% will be extended within 3 months from the date of receipt of the application from the family (legal heir). The family (legal heir) should approach this department for the release of the balance 50% of the relief which will be deposited in the bank in a joint account in the name of kin (legal heir) and the competent authority concerned. If no further inform

# Model Using Bag of Words

In [54]:
# Function that removes stop words and process the text

def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_word.append(lema_token)
    return " ".join(lema_word) 

In [55]:
# defining a function that returns response to query using bow

def chat_bow(text):
    s=stopword_(text)
    lemma=text_normalization(s) # calling the function to perform text normalization
    bow=cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value=cosine_value.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

In [56]:
chat_bow('Tell me about PM Kisan Samman Nidhi.')

"pm-kisan Objective The scheme aims to supplement the financial needs of all landholding farmers’ families by procuring various inputs to ensure proper crop health and appropriate yields, commensurate with the anticipated farm income as well as for domestic needs. Under the scheme an amount of ₹ 6000/- per year is released by the Central Government online directly into the bank accounts of the eligible farmers under Direct Benefit Transfer mode, subject to certain exclusions. Financial benefit of Rs. 6000 per annum per family payable in three equal installments of Rs 2000 each , every four months. All landholding farmers' families, which have cultivable land holding in their names are eligible to get benefit under the scheme. Step 1: The following are the prerequisites for the enrollment process: Aadhaar Card Landholding paper Savings bank account Step 2: The VLE will fill in the complete details of farmer registration details like State, district, subdistrict block, and village, key i

In [57]:
chat_bow('Who can apply for PMAY-G?')

'pfeslpbtac The scheme “Pension to the Persons Who Lost 100% Eye Sight" was launched by the Directorate of Social Welfare and Social Education, Government of Tripura. The scheme aims to provide a monthly pension of ₹2,000/- to the differently-abled persons who are 10 years and above in age and have 100% blindness. Pension of ₹2,000/- per month. The applicant should be a resident of Tripura. The age of the applicant should be 10 years & above. The applicant should be 100% Visually Handicapped. The family income of the applicant should not exceed ₹1,00,000/-. No member of the family of the applicant should be a government employee. Step 1: The applicant should visit the Office of the Child Development Project Officer, and request a hard copy of the prescribed format of the application form free of cost. OR The interested applicant should take print of the prescribed format of the application form. Step 2: In the application form, fill in all the mandatory fields (signed across, if requir

In [58]:
chat_bow('What benefits does PM KISAN scheme provide?')

'cbs-pbaocwwb The "Cash Benefit Scheme" is a Welfare Scheme by the Puducherry Building and Other Construction Workers Welfare Board, Labour Department, Union Territory of Puducherry. The objective of this scheme is to provide educational financial assistance to the wards of construction workers who have secured the top three ranks at the State Level and Regional Level in the Higher Secondary course or its equivalent examinations. The amount is credited through Direct Benefit Transfer (DBT) into the bank account of the nominee/ dependent. The applications are accepted offline. State-Level Cash Benefit (Onetime Payment) a) First Prize: ₹1,00,000. b) Second Prize: ₹80,000. c) Third Prize: ₹60,000. \ufeff Region Wise Cash Benefit (Onetime Payment) a) First Prize: ₹60,000. b) Second Prize: ₹40,000. c) Third Prize: ₹20,000. \ufeff Annual Educational Assistance a) Degree Courses: ₹2,000. b) Post Graduate Courses: ₹4,000. c) Diploma courses: ₹2,000. d) Postgraduate Diploma: ₹3,000. \ufeff Mode

# Model Using tf-idf

In [59]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

In [61]:
chat_tfidf('Is this scheme for rural or urban citizens?')

'dhkura The scheme “Distribution of Horticulture Kit to Urban and Rural Areas" was launched by the Department of Agriculture and Farmer\'s Welfare, Union Territory of Puducherry. Through this scheme, all the horticulture growers in rural & urban areas are provided with a horticulture kit worth ₹ 3,000 per kit to urban and rural areas, during Aadipattam/Thaipattam, through the Additional Director of Agriculture (Horticulture). 75% subsidy, i.e. distribution of horticulture kit worth ₹ 3,000 per kit to urban and rural areas (homestead/backyards), through the Additional Director of Agriculture (Horticulture), during Aadipattam/Thaipattam seasonality. All horticulture growers of rural & urban areas are eligible. Step 1: The interested applicant should visit (during office hours) the Uzhavar Udhaviyagam (Farmers\' Help Centres) or the Office of the Additional Director of Agriculture (Horticulture) and request the hard copy of the prescribed format of the application form from the Agricultur